In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix


In [3]:
# Cargar dataset desde CSV
csv_path = "dataset_labels.csv"
df = pd.read_csv(csv_path)

# Codificar etiquetas numéricamente
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])
num_classes = len(label_encoder.classes_)

# Dividir en train y val
train_df = df[df["split"] == "train"]
val_df = df[df["split"] == "val"]
test_df = df[df["split"] == "test"]


# Función para cargar imágenes
def load_images(df, img_size=(224, 224)):  # ResNet50 usa 224x224
    images = []
    labels = []
    for _, row in df.iterrows():
        img = cv2.imread(row["image_path"])
        img = cv2.resize(img, img_size)
        img = img.astype("float32") / 255.0  # Normalizar
        images.append(img)
        labels.append(row["label"])
    return np.array(images), to_categorical(labels, num_classes)

# Cargar imágenes
X_train, y_train = load_images(train_df)
X_val, y_val = load_images(val_df)
X_test, y_test = load_images(test_df)  # Agregado para testeo


# Cargar modelo preentrenado ResNet50
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Congelar capas preentrenadas

# Construcción del modelo
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
outputs = Dense(num_classes, activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=outputs)

# Compilar el modelo
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [4]:
# Entrenar el modelo
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32)

# Evaluar el modelo en el conjunto de testeo
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = np.argmax(y_test, axis=1)

# Calcular métricas
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average="weighted")
recall = recall_score(y_true, y_pred, average="weighted")
f1 = f1_score(y_true, y_pred, average="weighted")

# Matriz de confusión y reporte de clasificación
conf_matrix = confusion_matrix(y_true, y_pred)
report = classification_report(y_true, y_pred)

# Imprimir resultados
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", report)

Epoch 1/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 97s 2s/step - accuracy: 0.2383 - loss: 1.6816 - val_accuracy: 0.2488 - val_loss: 1.3798
Epoch 2/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 92s 2s/step - accuracy: 0.2909 - loss: 1.4590 - val_accuracy: 0.2778 - val_loss: 1.3463
Epoch 3/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 91s 2s/step - accuracy: 0.2932 - loss: 1.4080 - val_accuracy: 0.4010 - val_loss: 1.3232
Epoch 4/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 90s 2s/step - accuracy: 0.3068 - loss: 1.3619 - val_accuracy: 0.3527 - val_loss: 1.3227
Epoch 5/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 91s 2s/step - accuracy: 0.3246 - loss: 1.3360 - val_accuracy: 0.4372 - val_loss: 1.2994
6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step
Accuracy: 0.3901
Precision: 0.3207
Recall: 0.3901
F1-score: 0.3199
Confusion Matrix:
 [[34  0 15  1]
 [25  0 25  0]
 [18  0 29  3]
 [14  0 10  8]]
Classification Report:
               precision    recall  f1-score   support

           0       0.37      0.68      0.48        50
           1       0.00      0.00      0.00        50
    

c:\Users\dalil\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dalil\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\dalil\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo